# Preparacion del Entorno y Carga de Datos



In [2]:
# --- Importación de librerías y definir rutas---
import pandas as pd
import numpy as np 


# Ruta de los archivos
path = "./data/" # Usar mayúsculas para constantes es una convención común

In [3]:
# --- Carga de Datos ---
print("--- Iniciando Carga de Archivos ---")
try:
    charges_df = pd.read_csv(path + "Charges.csv")
    damages_df = pd.read_csv(path + "Damages.csv")
    endorse_df = pd.read_csv(path + "Endorse.csv")
    person_df = pd.read_csv(path + "Primary_Person.csv")
    restrict_df = pd.read_csv(path + "Restrict.csv")
    units_df = pd.read_csv(path + "Units.csv", low_memory=False)
    print("\n--- Carga Completada Exitosamente ---")
    dataframes_loaded = True # Bandera para saber si la carga fue exitosa

except FileNotFoundError as e:
    print(f"\nERROR FATAL: {e}. Verifica la ruta '{path}' y la existencia de los archivos.")
    dataframes_loaded = False
    # Detener o manejar el error como prefieras
except Exception as e:
    print(f"\nERROR INESPERADO durante la carga: {e}")
    dataframes_loaded = False
    # Detener o manejar el error como prefieras

# Lista de dataframes y sus nombres para usar en celdas posteriores (si la carga fue exitosa)
if dataframes_loaded:
    df_list = [
        ("charges_df", charges_df),
        ("damages_df", damages_df),
        ("endorse_df", endorse_df),
        ("person_df", person_df),
        ("restrict_df", restrict_df),
        ("units_df", units_df)
    ]
else:
    df_list = []
    print("\n*** Advertencia: La carga de datos falló, los análisis subsiguientes no se ejecutarán correctamente. ***")

--- Iniciando Carga de Archivos ---

--- Carga Completada Exitosamente ---


In [4]:
 #--- Exploración Inicial Rápida ---
if dataframes_loaded:
    print("\n--- Formas (Filas, Columnas) de los DataFrames Cargados ---")
    for name, df in df_list:
        print(f"{name}: {df.shape}")


    # info básica para uno o dos dataframes clave
    print("\n--- Información Detallada ( person_df) ---")
    person_df.info()
    print("\n--- Información Detallada (units_df) ---")
    units_df.info(verbose=True, show_counts=True)   
    print("\n--- Información Detallada (charges_df) ---")
    charges_df.info(verbose=True, show_counts=True) 
    print("\n--- Información Detallada (damages_df) ---")
    damages_df.info(verbose=True, show_counts=True)
    print("\n--- Información Detallada (endorse_df) ---")
    endorse_df.info(verbose=True, show_counts=True)
    print("\n--- Información Detallada (restrict_df) ---")
    restrict_df.info(verbose=True, show_counts=True)
else:
    print("\nSaltando Exploración Inicial: Dataframes no cargados.")


--- Formas (Filas, Columnas) de los DataFrames Cargados ---
charges_df: (116110, 5)
damages_df: (24950, 2)
endorse_df: (159818, 3)
person_df: (156954, 32)
restrict_df: (159852, 3)
units_df: (173499, 37)

--- Información Detallada ( person_df) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156954 entries, 0 to 156953
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CRASH_ID               156954 non-null  int64  
 1   UNIT_NBR               156954 non-null  int64  
 2   PRSN_NBR               156954 non-null  int64  
 3   PRSN_TYPE_ID           156954 non-null  object 
 4   PRSN_OCCPNT_POS_ID     156954 non-null  object 
 5   PRSN_INJRY_SEV_ID      156935 non-null  object 
 6   PRSN_AGE               154924 non-null  float64
 7   PRSN_ETHNICITY_ID      156680 non-null  object 
 8   PRSN_GNDR_ID           156935 non-null  object 
 9   PRSN_EJCT_ID           156935 non-null  object 
 10  

In [5]:
# Revisar si hay duplicados en los DataFrames


# Verificar si los DataFrames fueron cargados antes de realizar el análisis
if not dataframes_loaded:
    print("\nSaltando Chequeo de Duplicados: DataFrames no cargados.")
else:
    
    print("\n--- Chequeo de Duplicados Basado en Claves Lógicas ---")

    # Diccionario con las columnas clave para verificar duplicados en cada DataFrame
   
    keys_to_check = {
        'charges_df': ['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'CHARGE'],  
        'damages_df': None,
        'endorse_df': ['CRASH_ID', 'UNIT_NBR', 'DRVR_LIC_ENDORS_ID'],  
        'person_df': ['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR'], 
        'restrict_df': ['CRASH_ID', 'UNIT_NBR', 'DRVR_LIC_RESTRIC_ID'],  
        'units_df': ['CRASH_ID', 'UNIT_NBR'] 
    }

    # Convierte la lista de DataFrames en un diccionario para acceso rápido por nombre
    df_map = dict(df_list)

    # Itera sobre cada DataFrame y sus columnas clave definidas
    for name, key_cols in keys_to_check.items():
        # Obtiene el DataFrame por nombre (devuelve None si no existe)
        df = df_map.get(name)
        
        # Verifica si el DataFrame existe
        if df is None:
            print(f"\n--- Advertencia: DataFrame '{name}' no encontrado. Saltando. ---")
            continue  # Pasa al siguiente DataFrame

        print(f"\n--- Chequeando Duplicados en {name} ---")
        
        # Verifica si hay columnas clave definidas y si todas existen en el DataFrame
        if key_cols and all(col in df.columns for col in key_cols):
            # Busca duplicados basados en las columnas clave
            duplicates = df[df.duplicated(subset=key_cols, keep=False)]
            print(f"Clave usada: {key_cols}")
        else:
            # Si faltan columnas clave pero estaban definidas, muestra advertencia
            if key_cols:
                print(f"Advertencia: Columnas {key_cols} no encontradas. Usando filas completas.")
            # Busca duplicados en todas las columnas
            duplicates = df[df.duplicated(keep=False)]

        # Evalúa y muestra los resultados
        if duplicates.empty:
            print("✅ No hay duplicados.")
        else:
            print(f"⚠️ ALERTA: {len(duplicates)} filas duplicadas.")
            # Ordena por las columnas clave (o todas las columnas si no hay clave definida)
            print(duplicates.sort_values(by=key_cols if key_cols else df.columns.tolist()).head())


--- Chequeo de Duplicados Basado en Claves Lógicas ---

--- Chequeando Duplicados en charges_df ---
Clave usada: ['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'CHARGE']
⚠️ ALERTA: 509 filas duplicadas.
      CRASH_ID  UNIT_NBR  PRSN_NBR                                   CHARGE  \
1703  14852370         1         1  ILLEGAL U-TURN AT SIGNALED INTERSECTION   
1705  14852370         1         1  ILLEGAL U-TURN AT SIGNALED INTERSECTION   
1704  14852370         1         1                       NO DRIVERS LICENSE   
1706  14852370         1         1                       NO DRIVERS LICENSE   
2123  14855159         1         1                    FAIL TO CONTROL SPEED   

      CITATION_NBR  
1703   SO-EE762939  
1705   SO-EE762939  
1704   SO-EE762939  
1706   SO-EE762939  
2123  SOW-WW349467  

--- Chequeando Duplicados en damages_df ---
⚠️ ALERTA: 647 filas duplicadas.
     CRASH_ID          DAMAGED_PROPERTY
133  14846500     METAL FENCE & MAILBOX
134  14846500     METAL FENCE & MAILBOX
268  14

In [6]:
#  Eliminación de Duplicados


if not dataframes_loaded:
    print("\nSaltando Eliminación de Duplicados: DataFrames no cargados.")
    
else:
    print("\n--- Eliminando Duplicados Identificados ---")

    # Configuración centralizada de claves para deduplicación
    # None = eliminar duplicados completos (todas las columnas)
    keys_for_dedup = {
        'charges_df': ['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'CHARGE'],
        'damages_df':  None,
        'endorse_df': ['CRASH_ID', 'UNIT_NBR', 'DRVR_LIC_ENDORS_ID'],
        'person_df': ['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR'],
        'restrict_df': ['CRASH_ID', 'UNIT_NBR', 'DRVR_LIC_RESTRIC_ID'],
        'units_df': ['CRASH_ID', 'UNIT_NBR']
    }

    # Procesamiento unificado de todos los DataFrames
    original_shapes = {}
    new_shapes = {}
    
    for df_name, key_cols in keys_for_dedup.items():
        if df_name not in df_map:
            print(f"Advertencia: {df_name} no encontrado. Saltando.")
            continue
            
        df = df_map[df_name]
        original_shapes[df_name] = df.shape
        
        # Eliminación de duplicados (con o sin subset)
        df_map[df_name] = df.drop_duplicates(
            subset=key_cols if key_cols else None,  # None = todas las columnas
            keep='first'
        )
        
        new_shapes[df_name] = df_map[df_name].shape

    # --- Reporte consolidado ---
    print("\n--- Resumen de Eliminación de Duplicados ---")
    for df_name in original_shapes:
        orig, new = original_shapes[df_name], new_shapes.get(df_name)
        if new:
            removed = orig[0] - new[0]
            print(f"{df_name}: Filas eliminadas = {removed} (Original: {orig}, Nuevo: {new})")
        else:
            print(f"{df_name}: No se pudo procesar.")

    # Reasignación automática de variables (opcional)
    # Esto evita tener que actualizar manualmente cada variable
    globals().update({k: v for k, v in df_map.items() if k in keys_for_dedup})
    print("\nDataFrames actualizados en el namespace global.")


--- Eliminando Duplicados Identificados ---

--- Resumen de Eliminación de Duplicados ---
charges_df: Filas eliminadas = 268 (Original: (116110, 5), Nuevo: (115842, 5))
damages_df: Filas eliminadas = 344 (Original: (24950, 2), Nuevo: (24606, 2))
endorse_df: Filas eliminadas = 0 (Original: (159818, 3), Nuevo: (159818, 3))
person_df: Filas eliminadas = 0 (Original: (156954, 32), Nuevo: (156954, 32))
restrict_df: Filas eliminadas = 0 (Original: (159852, 3), Nuevo: (159852, 3))
units_df: Filas eliminadas = 5479 (Original: (173499, 37), Nuevo: (168020, 37))

DataFrames actualizados en el namespace global.


In [7]:
def handle_dataframe_errors(error):
    """Maneja errores comunes en operaciones con DataFrames"""
    if isinstance(error, KeyError):
        print(f"Error: {error}. Verifica el nombre de las columnas en el DataFrame.")
    elif isinstance(error, NameError):
        print("El DataFrame no está definido. Asegúrate de que se ha cargado correctamente.")
    else:
        print(f"Se produjo un error inesperado: {error}")

# Exploración 1: "¿Qué valores existen para género y severidad de lesión en los datos de personas?"

**Necesitamos:**
1. Verificar los valores únicos en `PRSN_GNDR_ID` (género de la persona)
2. Verificar los valores únicos en `PRSN_INJRY_SEV_ID` (severidad de la lesión)
3. Confirmar que el DataFrame `person_df` está cargado correctamente
4. Manejar posibles errores:
   - DataFrame no encontrado
   - Columnas no existentes
   - Errores inesperados

**Archivos requeridos:**
- Primary_Person.csv (cargado como `person_df`)

**Columnas clave:**
- `PRSN_GNDR_ID`: Identificador de género (ej: 'MALE', 'FEMALE')
- `PRSN_INJRY_SEV_ID`: Nivel de severidad de lesión (ej: 'NO INJURY', 'INCAPACITATING INJURY')

In [10]:
#Explorar los valores para el Analizis 1

print("--\n Valores de las columnas PRSN_GNDR_ID y PRSN_INJRY_SEV_ID en el archivo Primary_Person.csv-- \n")

#Verificar que person_df existe antes de usarlo
if 'person_df' in locals() or 'person_df' in globals():
    try:
        # Verificar los valores únicos de la columna PRSN_GNDR_ID
        print("Valores únicos de PRSN_GNDR_ID:")
        print(person_df['PRSN_GNDR_ID'].unique())

        # Verificar los valores únicos de la columna PRSN_INJRY_SEV_ID
        print("\nValores únicos de PRSN_INJRY_SEV_ID:")
        print(person_df['PRSN_INJRY_SEV_ID'].unique())
    
    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin de la exploración de valores--")

--
 Valores de las columnas PRSN_GNDR_ID y PRSN_INJRY_SEV_ID en el archivo Primary_Person.csv-- 

Valores únicos de PRSN_GNDR_ID:
['MALE' 'FEMALE' 'UNKNOWN' nan]

Valores únicos de PRSN_INJRY_SEV_ID:
['NOT INJURED' 'POSSIBLE INJURY' 'NON-INCAPACITATING INJURY' 'UNKNOWN'
 'INCAPACITATING INJURY' 'KILLED' nan]

--Fin de la exploración de valores--


In [11]:
#Analisis

print("Numero de choques donde hay algun  hombre fallecido:\n")
male_code = 'MALE'
killed_code = 'KILLED'

analy1_result = 0

#Volvemos a verificar que el DataFrame 'person_df' existe antes de usarlo
if 'person_df' in locals() or 'person_df' in globals():
    try:
        # Filtrar llso que son hombres y han muerto
        male_killed= person_df[
            (person_df['PRSN_GNDR_ID']==male_code) & 
            (person_df['PRSN_INJRY_SEV_ID']==killed_code)
        ]


#Crash_id para identificar almenos un hombre en el accident
        analy1_result=male_killed['CRASH_ID'].nunique()
        print(f"Numero de choques donde hay algun hombre fallecido: {analy1_result}")
         
    
    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin del primer Analisis--")

Numero de choques donde hay algun  hombre fallecido:

Numero de choques donde hay algun hombre fallecido: 180

--Fin del primer Analisis--


# Análisis 2: "Número total de vehículos de dos ruedas involucrados en choques"

**Necesitamos:**
1. Identificar vehículos de dos ruedas usando `VEH_BODY_STYL_ID`
2. Filtrar por códigos específicos: `['MOTORCYCLE', 'POLICE MOTORCYCLE']`
3. Contar el total de registros que cumplen el criterio



**Archivos requeridos:**
- Units.csv (cargado como `units_df`)

**Columnas clave:**
- `VEH_BODY_STYL_ID`: Tipo de vehículo (contiene los valores a filtrar)

In [13]:
#Explorar los valores para el Analizis 2

print("--\n Valores de las columnas VEH_BODY_STYL_ID en el archivo Units.csv-- \n")

#Verificar que units_df existe antes de usarlo
if 'units_df' in locals() or 'units_df' in globals():
    try:
        # Verificar los valores únicos de la columna VEH_BODY_STYL_ID
        print("Valores únicos de VEH_BODY_STYL_ID:")
        print(units_df['VEH_BODY_STYL_ID'].unique())
    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin de la exploración de valores--")

--
 Valores de las columnas VEH_BODY_STYL_ID en el archivo Units.csv-- 

Valores únicos de VEH_BODY_STYL_ID:
['PASSENGER CAR, 4-DOOR' 'TRUCK' nan 'SPORT UTILITY VEHICLE' 'PICKUP'
 'PASSENGER CAR, 2-DOOR' 'VAN' 'POLICE CAR/TRUCK'
 'OTHER  (EXPLAIN IN NARRATIVE)' 'MOTORCYCLE' 'UNKNOWN' 'BUS'
 'TRUCK TRACTOR' 'YELLOW SCHOOL BUS' 'FARM EQUIPMENT' 'FIRE TRUCK'
 'NEV-NEIGHBORHOOD ELECTRIC VEHICLE' 'AMBULANCE' 'POLICE MOTORCYCLE'
 'NOT REPORTED']

--Fin de la exploración de valores--


In [14]:
#Analisis 2
print("--Anlisis 2--")
print("Numero total de vehiculos de dos ruedas involucrados en choques:\n")

two_wheels_code = ['MOTORCYCLE', 'POLICE MOTORCYCLE']

analy2_result = 0

#Volvemos a verificar que el DataFrame 'units_df' existe antes de usarlo
if 'units_df' in locals() or 'units_df' in globals():
    try:
        # Filtrar los que son de dos ruedas
        two_wheels= units_df[
            (units_df['VEH_BODY_STYL_ID'].isin(two_wheels_code))
        ]
        #Contamos el numero de vehiculos de dos ruedas involucrados en choques
        analy2_result=len(two_wheels)

        print(f"Numero total de vehiculos de dos ruedas involucrados en choques: {analy2_result}")

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin del segundo Analisis--")

--Anlisis 2--
Numero total de vehiculos de dos ruedas involucrados en choques:

Numero total de vehiculos de dos ruedas involucrados en choques: 773

--Fin del segundo Analisis--


# Análisis 3: "Which state has the highest number of accidents in which females are involved?

Necesitamsp:
Filtrar person_df por PRSN_GNDR_ID == 'FEMALE'.
Agrupar el resultado por DRVR_LIC_STATE_ID.
Contar los CRASH_ID únicos para cada estado.
Identificar el estado con el conteo máximo.

Archivos: Primary_Person.csv (person_df)
Columnas: CRASH_ID, PRSN_GNDR_ID, DRVR_LIC_STATE_ID.

In [ ]:
# Valores para Análisis 3 

print("\n--- Explorando valores únicos para Análisis 3 ---")
print("Objetivo: Confirmar código 'FEMALE' y ver formato de 'DRVR_LIC_STATE_ID' en el archivo Primary_Person.csv (person_df)-.")

# Verificar que person_df existe
if 'person_df' in locals() or 'person_df' in globals():
    try:
        # Confirmar valor 'FEMALE' (ya lo sabemos, pero por formalidad)
        print("\nValores únicos en 'PRSN_GNDR_ID' (confirmación):")
        # Solo mostramos los únicos, ya sabemos que 'FEMALE' está
        print(person_df['PRSN_GNDR_ID'].unique()) 

        # Explorar la columna de estado de licencia
        print("\nPrimeros valores y conteo de valores únicos en 'DRVR_LIC_STATE_ID':")
        # Mostramos algunos valores para ver el formato
        print(person_df['DRVR_LIC_STATE_ID'].head().to_list()) 
        # Mostramos cuántos estados diferentes hay y los más comunes
        print(f"Número de estados únicos: {person_df['DRVR_LIC_STATE_ID'].nunique()}")
        print("\nEstados más comunes (Top 10):")
        print(person_df['DRVR_LIC_STATE_ID'].value_counts().head(10))

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin de la exploración para Análisis 3 ---")


--- Explorando valores únicos para Análisis 3 ---
Objetivo: Confirmar código 'FEMALE' y ver formato de 'DRVR_LIC_STATE_ID' en el archivo Primary_Person.csv (person_df)-.

Valores únicos en 'PRSN_GNDR_ID' (confirmación):
['MALE' 'FEMALE' 'UNKNOWN' nan]

Primeros valores y conteo de valores únicos en 'DRVR_LIC_STATE_ID':
['Texas', 'New York', 'Texas', 'Texas', 'Texas']
Número de estados únicos: 61

Estados más comunes (Top 10):
DRVR_LIC_STATE_ID
Texas         136335
Unknown         2268
Mexico          1007
Louisiana        752
New Mexico       699
California       596
Florida          544
Oklahoma         512
Other            352
Arkansas         344
Name: count, dtype: int64

--- Fin de la exploración para Análisis 3 ---


In [ ]:
#Analisis 3

print("--Analizis 3: Buscaremos el estado con más choques de mujeres:--\n")

famale_code = 'FEMALE'
drvr_lic_state_code = 'DRVR_LIC_STATE_ID'
analy3_count = 0

#Volvemos a verificar que el DataFrame 'person_df' existe antes de usarlo
if 'person_df' in locals() or 'person_df' in globals():
    try:
        # Filtrar los que son mujeres
        female_involved_df = person_df[
            (person_df['PRSN_GNDR_ID'] == famale_code) 
        ]        

        #agrupar por estado y contar los crash_id unicos
        crashes_by_state = female_involved_df.groupby(drvr_lic_state_code)['CRASH_ID'].nunique()
        # Ordernar los resultados de mayor a menor
        crashes_by_state_sorted = crashes_by_state.sort_values(ascending=False)

        

        print(f"\nEstado con más choques de mujeres: {crashes_by_state_sorted.index[0]}")
        print(f"Numero de choques: {crashes_by_state_sorted.iloc[0]}")

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin del tercer Analisis 3--")

--Analizis 3: Buscaremos el estado con más choques de mujeres:--


Estado con más choques de mujeres: Texas
Numero de choques: 42138

--Fin del tercer Analisis 3--


# Análisis 4: "Which are the Top 5th to 15th VEH_MAKE_IDs that contribute to a largest number of injuries including death"

**Necesitamos:**
1. Filtrar personas con lesiones/muertes usando `PRSN_INJRY_SEV_ID` con códigos:
   - `POSSIBLE INJURY`
   - `NON-INCAPACITATING INJURY`
   - `INCAPACITATING INJURY`
   - `KILLED`
2. Unir con `units_df` para obtener la marca del vehículo (`VEH_MAKE_ID`)
3. Contar lesiones por marca de vehículo
4. Ordenar y seleccionar posiciones 5 a 15


**Archivos requeridos:**
- `Primary_Person.csv` (como `person_df`)
- `Units.csv` (como `units_df`)

**Columnas clave:**
- En `person_df`:
  - `CRASH_ID`: Identificador del accidente
  - `UNIT_NBR`: Número de unidad vehicular
  - `PRSN_INJRY_SEV_ID`: Severidad de la lesión
- En `units_df`:
  - `CRASH_ID`: Identificador del accidente
  - `UNIT_NBR`: Número de unidad vehicular
  - `VEH_MAKE_ID`: Marca del vehículo

In [ ]:
# Explorar los valores para el Analizis 4

print("--Valores en PRSN_INJRY_SEV_ID  en Primary_Person.csv--\n")

# Verificar que person_df existe antes de usarlo
if 'person_df' in locals() or 'person_df' in globals():
    try:
        print(person_df['PRSN_INJRY_SEV_ID'].unique())

        print(person_df['PRSN_INJRY_SEV_ID'].value_counts())

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin de la exploracion para el Analisis 4--")    

--Valores en PRSN_INJRY_SEV_ID  en Primary_Person.csv--

['NOT INJURED' 'POSSIBLE INJURY' 'NON-INCAPACITATING INJURY' 'UNKNOWN'
 'INCAPACITATING INJURY' 'KILLED' nan]
PRSN_INJRY_SEV_ID
NOT INJURED                  120752
POSSIBLE INJURY               18752
NON-INCAPACITATING INJURY     11850
INCAPACITATING INJURY          2674
UNKNOWN                        2661
KILLED                          246
Name: count, dtype: int64

--Fin de la exploracion para el Analisis 4--


In [ ]:
#Analisis 4


print("--Analizis 4: Top 5-15 de marcas de vehiculos (VEH_MAKE_ID) por numero de lesions incluyendo la muerte--\n") 


injury_code = ['POSSIBLE INJURY', 'NON-INCAPACITATING INJURY', 'INCAPACITATING INJURY', 'KILLED'] 

analy4_result = pd.DataFrame() 

#Volvemos a verificar que el DataFrame 'units_df' y "person_df" existe antes de usarlo

if ('units_df' in locals() or 'units_df' in globals()) and ('person_df' in locals() or 'person_df' in globals()):
    try:
   
        injured_person = person_df[
            person_df['PRSN_INJRY_SEV_ID'].isin(injury_code)
        ]
        print(f"Numero de personas lesionadas o muertas: {len(injured_person)}")

      
        # Unir los DataFrames por CRASH_ID 
        merged_df = pd.merge(injured_person, units_df, on=['CRASH_ID', 'UNIT_NBR'], how='inner')
       
        # Agrupar por VEH_MAKE_ID y contar el número de lesiones
        injury_count = merged_df.groupby('VEH_MAKE_ID')['PRSN_INJRY_SEV_ID'].count() 
      

        injury_count_sorted = injury_count.sort_values(ascending=False)

        
        # Obtener el top 5-15 
        top_injuries = injury_count_sorted.iloc[4:15].reset_index()
        top_injuries.index = range(5, 16)  # Índice del 5 al 15

        print("\nTop 5-15 de marcas de vehículos por número de lesiones incluyendo la muerte:")
        print(top_injuries)

 
    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin del Analisis 4--")

--Analizis 4: Top 5-15 de marcas de vehiculos (VEH_MAKE_ID) por numero de lesions incluyendo la muerte--

Numero de personas lesionadas o muertas: 33522

Top 5-15 de marcas de vehículos por número de lesiones incluyendo la muerte:
   VEH_MAKE_ID  PRSN_INJRY_SEV_ID
5       NISSAN               2441
6        HONDA               2312
7          GMC                935
8      HYUNDAI                900
9          KIA                822
10        JEEP                817
11    CHRYSLER                708
12       MAZDA                558
13     PONTIAC                471
14  VOLKSWAGEN                468
15       LEXUS                428

--Fin del Analisis 4--


# Análisis 5: "Top 5 Zip Codes (Driver Zip Code) with the highest number of crashes with alcohol as the contributing factor"

**Necesitamos:**
1. Identificar accidentes donde el alcohol fue factor usando:
   - `CONTRIB_FACTR_1_ID` o `CONTRIB_FACTR_2_ID` con valores:
     - `UNDER INFLUENCE - ALCOHOL`
     - `HAD BEEN DRINKING`
2. Filtrar solo conductores (`PRSN_TYPE_ID = 'DRIVER'`)
3. Extraer y limpiar códigos postales (`DRVR_ZIP`) conservando solo los 5 primeros dígitos
4. Contar accidentes únicos por código postal
5. Seleccionar los 5 códigos postales con más incidentes
6. Manejar errores de:
   - Datos faltantes
   - Formatos incorrectos
   - DataFrames no cargados

**Archivos requeridos:**
- `Units.csv` (como `units_df`)
- `Primary_Person.csv` (como `person_df`)

**Columnas clave:**
- En `units_df`:
  - `CRASH_ID`: Identificador único del accidente
  - `UNIT_NBR`: Número de unidad vehicular
  - `CONTRIB_FACTR_1_ID`/`2_ID`: Factores contribuyentes
- En `person_df`:
  - `PRSN_TYPE_ID`: Tipo de persona (debe incluir 'DRIVER')
  - `DRVR_ZIP`: Código postal del conductor

In [ ]:
# Explorar los valores para el Analizis 5

print("--Indentificar el codigo para el alcohol en 'PRSN_ALC_RSLT_ID'  y ver formato del zip codo en 'PRSN_ZIP_ID' en el archivo 'Primary_person'--\n")

# Verificar que person_df existe antes de usarlo
if ('person_df' in locals() or 'person_df' in globals()) and \
    ('units_df' in locals() or 'units_df' in globals()):
    try:

        #exploramos la columna del alcohol
        print("Valores únicos en 'PRSN_ALC_RSLT_ID':")
        print(person_df['PRSN_ALC_RSLT_ID'].unique())

        #exploramos la columna del zip
        print("\nPrimeros valores y conteo de valores únicos en 'PRSN_ZIP_ID':")
        # Mostramos algunos valores para ver el formato
        print(person_df['DRVR_ZIP'].head().to_list())

        #exploramos la columna CONTRIB_FACTR_1_ID
        print("\nPrimeros valores y conteo de valores únicos en 'CONTRIB_FACTR_1_ID':")
        # Mostramos algunos valores para ver el formato 
        print(units_df['CONTRIB_FACTR_1_ID'].unique())

        #Exploramos la columna CONTRIB_FACTR_2_ID
        print("\nPrimeros valores y conteo de valores únicos en 'CONTRIB_FACTR_2_ID':")
        # Mostramos algunos valores para ver el formato
        print(units_df['CONTRIB_FACTR_2_ID'].unique)

        #Exploramos la columna CONTRIB_FACTR_P1_ID  
        print("\nValores únicos en 'CONTRIB_FACTR_P1_ID':")
        # Mostramos algunos valores para ver el formato
        print(units_df['CONTRIB_FACTR_P1_ID'].unique)

        print(f"Número de valores nulos (NaN): {person_df['DRVR_ZIP'].isna().sum()}")
        print(person_df['DRVR_ZIP'].dropna().astype(str).head(10).to_list())

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin de la exploración para Análisis 5 ---")

--Indentificar el codigo para el alcohol en 'PRSN_ALC_RSLT_ID'  y ver formato del zip codo en 'PRSN_ZIP_ID' en el archivo 'Primary_person'--

Valores únicos en 'PRSN_ALC_RSLT_ID':
['Positive' nan 'Negative']

Primeros valores y conteo de valores únicos en 'PRSN_ZIP_ID':
['77357', '13830', '78934', '76520', '76707']

Primeros valores y conteo de valores únicos en 'CONTRIB_FACTR_1_ID':
['UNDER INFLUENCE - ALCOHOL' 'HAD BEEN DRINKING' 'NONE'
 'DISREGARD STOP AND GO SIGNAL' 'FAILED TO CONTROL SPEED'
 'FAILED TO DRIVE IN SINGLE LANE' 'UNSAFE SPEED' 'FATIGUED OR ASLEEP'
 'CHANGED LANE WHEN UNSAFE' 'DRIVER INATTENTION' nan
 'FAULTY EVASIVE ACTION' 'TURNED IMPROPERLY - WIDE RIGHT'
 'FOLLOWED TOO CLOSELY' 'TURNED IMPROPERLY - WRONG LANE'
 'TURNED WHEN UNSAFE' 'FAILED TO YIELD ROW - STOP SIGN'
 'FAILED TO YIELD ROW - TURNING LEFT'
 'TURNED IMPROPERLY - CUT CORNER ON LEFT'
 'IMPAIRED VISIBILITY (EXPLAIN IN NARRATIVE)' 'PASSED IN NO PASSING LANE'
 'FAILED TO YIELD ROW - OPEN INTERSECTION'
 'FAILED

In [ ]:
# --- Análisis 5

print("--- Análisis 5: Top 5 Driver ZIP Codes (Alcohol como Factor Contribuyente) ---\n")


# Códigos para Factor Contribuyente de Alcohol (Confirmados)
ALCOHOL_FACTOR_CODES = ['UNDER INFLUENCE - ALCOHOL', 'HAD BEEN DRINKING']
# Código para Tipo de Persona = Conductor 
DRIVER_PERSON_CODE = 'DRIVER' 

# Variables para resultado
analysis_5_result = pd.Series(dtype=int) # Usaremos una Serie para el resultado

# Verificar que los DataFrames necesarios existan
if ('units_df' in locals() or 'units_df' in globals()) and \
   ('person_df' in locals() or 'person_df' in globals()):
    try:
       #  Filtrar unidades
        units_alcohol = units_df[
            units_df['CONTRIB_FACTR_1_ID'].isin(ALCOHOL_FACTOR_CODES) |
            units_df['CONTRIB_FACTR_2_ID'].isin(ALCOHOL_FACTOR_CODES)
        ][['CRASH_ID', 'UNIT_NBR']].drop_duplicates()

        #  Filtrar conductores
        if DRIVER_PERSON_CODE not in person_df['PRSN_TYPE_ID'].unique():
            raise ValueError(f"Código de conductor '{DRIVER_PERSON_CODE}' no válido.")

        drivers_df = person_df[
            person_df['PRSN_TYPE_ID'] == DRIVER_PERSON_CODE
        ][['CRASH_ID', 'UNIT_NBR', 'DRVR_ZIP']]

        #  Unir ambos
        merged_df = pd.merge(units_alcohol, drivers_df, on=['CRASH_ID', 'UNIT_NBR'])

        #  Limpiar ZIP
        merged_df['DRVR_ZIP'] = merged_df['DRVR_ZIP'].astype(str).str.extract(r'^(\d{5})')[0]
        merged_df.dropna(subset=['DRVR_ZIP'], inplace=True)

        #  Agrupar y contar
        zip_crash_counts = merged_df.groupby('DRVR_ZIP')['CRASH_ID'].nunique()

        #  Top 5
        if not zip_crash_counts.empty:
            analysis_5_result = zip_crash_counts.sort_values(ascending=False).head(5)

            # Resetear el índice para que sea 1, 2, 3, 4, 5 visualmente
            top_5_zip_codes = analysis_5_result.reset_index()
            top_5_zip_codes.index = range(1,6)

            print("\n--- Resultado Análisis 5 ---")
            print(top_5_zip_codes )

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin del Análisis 5 ---")

--- Análisis 5: Top 5 Driver ZIP Codes (Alcohol como Factor Contribuyente) ---


--- Resultado Análisis 5 ---
  DRVR_ZIP  CRASH_ID
1    76010        54
2    78521        52
3    78741        44
4    75067        44
5    78550        42

--- Fin del Análisis 5 ---


# Análisis 6: "Count of Distinct Crash IDs where No Damaged Property was observed and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance."

**Necesitamos:**
1. Identificar choques que cumplan **tres condiciones simultáneas**:
   - **Sin daño a propiedad** (no presentes en `damages_df`)
   - **Con daño vehicular severo** (niveles 5-7 en `VEH_DMAG_SCL_1_ID` o `VEH_DMAG_SCL_2_ID`)
   - **Con seguro válido** (según `FIN_RESP_TYPE_ID`)

**Archivos requeridos:**
- `Units.csv` (como `units_df`)
- `Damages.csv` (como `damages_df`)

**Columnas clave:**
- En `units_df`:
  - `CRASH_ID`: Identificador único del accidente
  - `VEH_DMAG_SCL_1_ID`/`VEH_DMAG_SCL_2_ID`: Escala de daño vehicular
  - `FIN_RESP_TYPE_ID`: Tipo de seguro/responsabilidad financiera
- En `damages_df`:
  - `CRASH_ID`: Identificador del accidente (para detectar daños a propiedad)

In [ ]:
# Explorar los valores para el Analizis 6

print("--Identificar los codigos para propiedad no danificada, Danio mayor que 4 y que el carro tenga seguro--\n")

# Verificar que units_df existe antes de usarlo
if ('damages_df' in locals() or 'damages_df' in globals()) and \
   ('units_df' in locals() or 'units_df' in globals()):
    

    try:
        #exploramos la columna de daños 'DAMAGED_PROPERTY' en damages_df
        print("Valores únicos en 'DAMAGED_PROPERTY':")
        damage_property_values=print(damages_df['DAMAGED_PROPERTY'].unique().tolist())
        # Mostramos cuántos valores únicos hay
        print(damage_property_values)
        

        #exploramos la columna con los niveles de daños 1 'VEH_DMAG_SCL_1_ID' en units_df    
        print("\nValores únicos en 'VEH_DMAG_SCL_1_ID':")
        print(units_df['VEH_DMAG_SCL_1_ID'].unique())
        # Mostramos cuántos valores únicos hay


        #exploramos la columna con los niveles de daños 2 'VEH_DMAG_SCL_2_ID' en units_df   
        print("\nValores únicos en 'VEH_DMAG_SCL_2_ID':")
        print(units_df['VEH_DMAG_SCL_2_ID'].unique())       

        #explorammos la columna de seguro 'FIN_RESP_TYPE_ID' en units_df
        print("\nValores únicos en 'FIN_RESP_TYPE_ID':")
        print(units_df['FIN_RESP_TYPE_ID'].unique())

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin de la voloracion de los valores del Analisis 6--")


--Identificar los codigos para propiedad no danificada, Danio mayor que 4 y que el carro tenga seguro--

Valores únicos en 'DAMAGED_PROPERTY':
['MAILBOX', 'YARD, GRASS', 'GUARDRAIL', 'ROAD SIGN', '2009 MAZDA 3', 'CHAIN LINK FENCE', 'WOODED POLE ON SOUTH SIDE OF LOOP', 'CITY SIGN FOR TURN LANES', 'FENCE-CHAIN LINK', 'LANDSCAPING AND METAL SIGN', 'APARTMENT BUILDING', 'STREET SIGN', 'HOUSE', 'LIGHT POLE', 'MINOR DAMAGE TO WOODEN POLE', 'CITY OF SAN ANTONIO ALLAN WARRICK SIGN', 'METAL POLES', 'WATER ATTENUATOR', 'UTILITY POST', '30 FEET OF GUARDRAIL', 'SCHOOL ZONE SIGN POLE', 'WOODEN FENCE', 'CONCRETE BARRIER', 'FENCE DAMAGE', 'STEEL FENCE', 'SIGNAL LIGHT CONTROL BOX', 'UTILITY POLE/HIGHLINE', 'TRASHCANS (2)', 'MAILBOX, TREE, CULVERT', 'FENCE', 'TREE', 'METAL LIGHT POLE', 'BARBED WIRE FENCE AND POSTS', 'WATER CRASH BARRIER', 'SIGN', 'GAS METER', 'US POSTAL BOX', 'TRAFFIC LITE POLE', 'UTILITY POLE', 'STREET LIGHT POLE', 'LEFT FREEWAY BARRIER', 'RIGHT FREEWAY BARRIER', 'FENCE DOOR', 'TELEPH

In [ ]:
#Analisis 6

print("--Contar Crash IDs únicos: Sin Daño Propiedad & Daño Vehículo > 4 & Con Seguro.--\n")

# Definimos los códigos para los filtros


scale_code = ['DAMAGED 5', 'DAMAGED 6', 'DAMAGED 7 HIGHEST']
insurence_code = [
    'PROOF OF LIABILITY INSURANCE', 
    'LIABILITY INSURANCE POLICY',
    'INSURANCE BINDER', 
    'CERTIFICATE OF SELF-INSURANCE', 
    'SURETY BOND', 
    'CERTIFICATE OF DEPOSIT WITH COMPTROLLER',
    'CERTIFICATE OF DEPOSIT WITH COUNTY JUDGE'
]

analy6_result = 0

#Volvemos a verificar que el DataFrame 'units_df' y "damages_df" existe antes de usarlo
if ('damages_df' in locals() or 'damages_df' in globals()) and \
   ('units_df' in locals() or 'units_df' in globals()):
    
    try:
        #  Obtener TODOS los CRASH_ID únicos de units_df (choques registrados)
        all_crashes = set(units_df['CRASH_ID'].unique())
        print(f"Número total de choques registrados: {len(all_crashes)}")

        # Choques CON daño a propiedad (damages_df)
        crashes_with_property_damage = set(damages_df['CRASH_ID'].unique())
        print(f"Choques CON daño a propiedad: {len(crashes_with_property_damage)}")

        # Choques SIN daño a propiedad (restar conjuntos)
        crashes_no_property_damage = all_crashes - crashes_with_property_damage
        print(f"Choques SIN daño a propiedad: {len(crashes_no_property_damage)}")

        # Choques con daño > 4 y asegurados (units_df)
        high_damage_insured = units_df[
            (
                (units_df['VEH_DMAG_SCL_1_ID'].isin(scale_code)) |
                (units_df['VEH_DMAG_SCL_2_ID'].isin(scale_code))
            ) &
            (units_df['FIN_RESP_TYPE_ID'].isin(insurence_code))
        ]
        high_damage_set = set(high_damage_insured['CRASH_ID'].unique())
        print(f"Choques con daño > 4 y asegurados: {len(high_damage_set)}")

        # Choques que cumplen AMBAS condiciones (intersección)
        final_crashes = crashes_no_property_damage & high_damage_set
        analy6_result = len(final_crashes)
        print(f"Resultado final (choques sin daño a propiedad + daño > 4 + seguro): {analy6_result}")
      
         
    
    except Exception as e:
         handle_dataframe_errors(e)

print("\n--Fin del Analisis 6--")

--Contar Crash IDs únicos: Sin Daño Propiedad & Daño Vehículo > 4 & Con Seguro.--

Número total de choques registrados: 83805
Choques CON daño a propiedad: 20887
Choques SIN daño a propiedad: 62918
Choques con daño > 4 y asegurados: 12939
Resultado final (choques sin daño a propiedad + daño > 4 + seguro): 8852

--Fin del Analisis 6--


#  Analisi 7: Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers, uses top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data


**Flujo de análisis:**
```mermaid
graph TD
    A[Top 25 estados<br>(units_df.VEH_LIC_STATE_ID)] --> B[Filtro inicial]
    C[Top 10 colores<br>(units_df.VEH_COLOR_ID)] --> B
    D[Conductores con licencia<br>(person_df)] --> B
    E[Infracciones speeding<br>(charges_df)] --> B
    B --> F[Conteo por marca<br>(units_df.VEH_MAKE_ID)]
```

**Archivos:**  
`Charges.csv` (`charges_df`),  
`Primary_Person.csv` (`person_df`),  
`Units.csv` (`units_df`).

**Columnas:**  
- `charges_df`: `CRASH_ID`, `PRSN_NBR`, `CHARGE`  
- `person_df`: `CRASH_ID`, `PRSN_NBR`, `UNIT_NBR`, `DRVR_LIC_TYPE_ID`, `DRVR_LIC_STATE_ID`  
- `units_df`: `CRASH_ID`, `UNIT_NBR`, `VEH_COLOR_ID`, `VEH_MAKE_ID`, `VEH_LIC_STATE_ID`


In [ ]:
#Explorar los valores para el Analizis 7

print("\n--Identificar valores para Speeding, license Driver, Color, vehiculo state--\n")

# Verificar Dataframes
if ('charges_df' in locals() or 'charges_df' in globals()) and \
   ('person_df' in locals() or 'person_df' in globals()) and \
   ('units_df' in locals() or 'units_df' in globals()):
    
 

    try:
        #exploramos la columna de Speeding 'CHARGE_DESC' en charges_df
        print("Valores únicos en 'CHARGE':")
        speeding_values=print(charges_df['CHARGE'].unique().tolist())
        # Mostramos cuántos valores únicos hay
        print(speeding_values)

        #exploramos la columna de licencia 'DRVR_LIC_TYPE_ID' en person_df    
        print("\nValores únicos en 'DRVR_LIC_TYPE_ID':")
        print(person_df['DRVR_LIC_TYPE_ID'].unique())

        #exploramos la columna de color 'VEH_COLOR_ID' en units_df
        print("\nValores únicos en 'VEH_COLOR_ID':")
        print(units_df['VEH_COLOR_ID'].unique())

        #exploramos la columna de estado 'VEH_STATE_ID' en units_df
        print("\nValores únicos en 'VEH_LIC_STATE_ID':")
        print(units_df['VEH_LIC_STATE_ID'].unique())
        # Mostramos cuántos valores únicos hay

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin de la exploración para Análisis 7 ---")


--Identificar valores para Speeding, license Driver, Color, vehiculo state--

Valores únicos en 'CHARGE':
['DRIVING WHILE INTOXICATED', 'DWI', 'RAN RED LIGHT SOLID (TC 544.007)', "NO DRIVER'S LICENSE (TC521.025)", 'DWI W/BAC >.015', 'FAILED TO DRIVE SINGLE LANE, NOL, NO INS', 'NO DRIVERS LICENSE', 'FAIL TO CONTROL SPEED', 'LEAVING THE SCENE', 'FTLI   / DWLI', 'LEAVING SCENE OF ACCIDENT', 'FAIL TO DRIVE IN SINGLE LANE', 'FAIL TO REPORT ACCIDENT', 'FTLI', 'FAIL TO MAINTAIN SINGLE LANE', 'POSS PG 3 UNDER 28', 'POSS MJ UNDER 2', 'UNSAFE SPEED', 'NO VALID OPERATOR LICENSE', 'NO VALID LIABILITY INSURANCE', 'CRIMINAL MISCHIEF $2500 - $30000', 'EVADING IN VEHICLE', 'CRIMINAL MISHCHIEF OVER $2500', 'EVADING IN  A VEHICLE', 'DWI OVER .15', 'NO OPERATING LICENSE', 'NO OPERATORS LICENSE', 'DRIVING WHILE LICENSE INVALID', 'FAIL TO REPORT NON INJURY ACCIDENT AT ONCE  TO  AUTHORITIES', "NO DRIVER'S LICENSE", 'FAILED TO CONTROL SPEED ACCIDENT', 'DRIVING WHILE  INTOXICATED', 'FAIL TO MAINTAIN FINANCIA

In [ ]:
# Calcular Top 25 estados por ofensas de velocidad

print("\n--Top 25 estados por ofensas de velocidad--\n")

top_25_states_offenses = []
states_to_exclude = ['Unknown', 'Mexico', 'Other'] # Lista de valores a excluir

#Verificar Data frames
if ('charges_df' in locals() or 'charges_df' in globals()) and \
   ('person_df' in locals() or 'person_df' in globals()): 
    try:
        #  Unir Charges y Person 
        person_subset = person_df[['CRASH_ID', 'PRSN_NBR', 'DRVR_LIC_STATE_ID']].copy()
        charges_with_state = pd.merge(
            charges_df[['CRASH_ID', 'CHARGE','PRSN_NBR']], 
            person_subset, 
            on=['CRASH_ID', 'PRSN_NBR'],        
            how='inner'
        )
        print(f"Numero de ofensas unidas con estado: {len(charges_with_state)}") 

        #  Agrupar por Estado y Contar Cargos 
        offenses_by_state = charges_with_state.groupby('DRVR_LIC_STATE_ID').size()
        
        # --- 3. Excluir No-Estados ANTES de ordenar ---
        offenses_by_us_state = offenses_by_state[~offenses_by_state.index.isin(states_to_exclude)]
        print(f"Número de estados/entidades después de excluir {states_to_exclude}: {len(offenses_by_us_state)}")

        # Ordenar 
        offenses_by_us_state_sorted = offenses_by_us_state.sort_values(ascending=False)
        
        # Seleccionar Top 25 Estados de EE. UU. 
        top_25_us_states_series = offenses_by_us_state_sorted.head(25)
        
        # Obtener la lista de nombres de estado (el índice de la Serie)
        top_25_states_list = top_25_us_states_series.index.tolist() # <--- ESTA ES LA LISTA QUE NECESITAMOS

        print("\nTop 25 estados por ofensas de velocidad:")
        print(top_25_states_list)

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin del Top 25 de estados por ofensa ---")



--Top 25 estados por ofensas de velocidad--

Numero de ofensas unidas con estado: 222372
Número de estados/entidades después de excluir ['Unknown', 'Mexico', 'Other']: 58

Top 25 estados por ofensas de velocidad:
['Texas', 'Louisiana', 'New Mexico', 'California', 'Florida', 'Oklahoma', 'Arkansas', 'Arizona', 'Georgia', 'Colorado', 'Illinois', 'Mississippi', 'Missouri', 'Tennessee', 'North Carolina', 'Kansas', 'Alabama', 'Michigan', 'Washington', 'New York', 'Ohio', 'Virginia', 'South Carolina', 'Indiana', 'Pennsylvania']

--- Fin del Top 25 de estados por ofensa ---


In [ ]:
#TOP 10 de colores por vehiculos

print("\n--Top 10 colores por vehiculos--\n")

top_10_colors_list = []

#Verificar Data framame units_df
if ('units_df' in locals() or 'units_df' in globals()):
    try:
        # Agrupar por color y contar cuántos vehículos hay de cada color
        color_counts = units_df['VEH_COLOR_ID'].value_counts(dropna=True)
        

        #colores a excluir
        codes_to_exclude = ['99', '98', 'NA', 'UNKNOWN', 'OTHER', 'MUL', 'ZZZ', 'NOT REPORTED'] 
        valid_color_counts = color_counts[~color_counts.index.isin(codes_to_exclude)]
        print(f"Número de códigos de color después de excluir genéricos: {len(valid_color_counts)}")

        # Obtener el top 10
        top_10_colors_series = color_counts.head(10)
        
        # Obtener solo la lista de colores (el índice de la Serie)
        top_10_colors_list = top_10_colors_series.index.tolist()
        
        print(f"\nResultado del Cálculo:")
        print("Top 10 Colores por número de vehículos:")
        print(top_10_colors_series) # Muestra los colores y sus conteos

       

    except Exception as e:
         handle_dataframe_errors(e)

print("\n--- Fin del Top 10 de colores por vehículos ---")  


--Top 10 colores por vehiculos--

Número de códigos de color después de excluir genéricos: 21

Resultado del Cálculo:
Top 10 Colores por número de vehículos:
VEH_COLOR_ID
WHI    38354
BLK    27749
SIL    20777
GRY    18174
BLU    15471
RED    14095
GRN     6767
MAR     6010
TAN     4846
GLD     4062
Name: count, dtype: int64

--- Fin del Top 10 de colores por vehículos ---


In [86]:
# --- Mapeo de Nombres de Estado a Abreviaturas (Top 25) ---

# Lista obtenida del Paso 15 (Revisado)
top_25_states_list = ['Texas', 'Louisiana', 'New Mexico', 'California', 'Florida', 'Oklahoma', 'Arkansas', 'Arizona', 'Georgia', 'Colorado',
                       'Illinois', 'Mississippi', 'Missouri', 'Tennessee', 'North Carolina', 'Kansas', 'Alabama', 'Michigan', 'Washington', 
                       'New York', 'Ohio', 'Virginia', 'South Carolina', 'Indiana', 'Pennsylvania']
# Diccionario completo para mapear nombres a abreviaturas
state_name_to_abbr = {
    'Texas': 'TX',
    'Louisiana': 'LA',
    'New Mexico': 'NM',
    'California': 'CA',
    'Florida': 'FL',
    'Oklahoma': 'OK',
    'Arkansas': 'AR',
    'Arizona': 'AZ',
    'Georgia': 'GA',
    'Colorado': 'CO',
    'Illinois': 'IL',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Tennessee': 'TN',
    'North Carolina': 'NC',
    'Kansas': 'KS',
    'Alabama': 'AL',
    'Michigan': 'MI',
    'Washington': 'WA',
    'New York': 'NY',
    'Ohio': 'OH',
    'Virginia': 'VA',
    'South Carolina': 'SC',
    'Indiana': 'IN',
    'Pennsylvania': 'PA'
}

print(f"Diccionario de mapeo creado con {len(state_name_to_abbr)} estados.")

# Crear la lista de abreviaturas correspondientes a los Top 25 estados
# Usamos list comprehension verificando que el estado esté en el diccionario (deberían estar todos)
top_25_state_abbrs_list = [state_name_to_abbr[state] for state in top_25_states_list if state in state_name_to_abbr]

print(f"Número de abreviaturas obtenidas para Top 25: {len(top_25_state_abbrs_list)}")
# Debería imprimir 25 si todos los nombres estaban en el diccionario

print("\nLista de abreviaturas de los Top 25 Estados (para filtrar VEH_LIC_STATE_ID):")
print(top_25_state_abbrs_list) # ['TX', 'LA', 'NM', 'CA', ..., 'PA']

Diccionario de mapeo creado con 25 estados.
Número de abreviaturas obtenidas para Top 25: 25

Lista de abreviaturas de los Top 25 Estados (para filtrar VEH_LIC_STATE_ID):
['TX', 'LA', 'NM', 'CA', 'FL', 'OK', 'AR', 'AZ', 'GA', 'CO', 'IL', 'MS', 'MO', 'TN', 'NC', 'KS', 'AL', 'MI', 'WA', 'NY', 'OH', 'VA', 'SC', 'IN', 'PA']


In [ ]:
# --- PASO 18: Realizar Análisis 7 (Cálculo Final) ---

print("\n--- Iniciando Análisis 7 ---")
print("Objetivo: Top 5 Marcas (Velocidad, Licencia Válida, Color Top 10, Estado Matrícula Top 25).")

# --- Listas y Códigos Definidos Previamente ---
# Asumimos que estas listas existen de pasos anteriores:
# top_10_colors_list = ['WHI', 'BLK', ...] 
# top_25_state_abbrs_list = ['TX', 'LA', ...


VALID_LICENSE_CODES = ['DRIVER LICENSE', 'COMMERCIAL DRIVER LIC.', 'OCCUPATIONAL'] # Confirmado en Paso 17

analysis7_result = pd.DataFrame() # Inicializar

# Verificar DataFrames y listas necesarias
if ('charges_df' in locals() or 'charges_df' in globals()) and \
   ('person_df' in locals() or 'person_df' in globals()) and \
   ('units_df' in locals() or 'units_df' in globals()) and \
   ('top_10_colors_list' in locals() or 'top_10_colors_list' in globals()) and \
   ('top_25_state_abbrs_list' in locals() or 'top_25_state_abbrs_list' in globals()):
    
    if not top_10_colors_list or not top_25_state_abbrs_list:
         print("ERROR: Las listas 'top_10_colors_list' o 'top_25_state_abbrs_list' están vacías. Ejecuta los pasos previos.")
    else:
        try:
            # Filtrar Charges por Velocidad 
            charges_df['CHARGE_STR'] = charges_df['CHARGE'].astype(str) 
            speeding_charges = charges_df[
                # Contiene 'SPEED' (ignorando mayúsculas/minúsculas) O es 'FAIL TO CONTROL SPEED'
                charges_df['CHARGE_STR'].str.contains('SPEED', case=False, na=False) |
                (charges_df['CHARGE_STR'] == 'FAIL TO CONTROL SPEED') 
            ][['CRASH_ID', 'PRSN_NBR']]
            print(f"Número de cargos por velocidad/control encontrados: {len(speeding_charges)}")

            # Filtrar Person por Licencia Válida 
            licensed_drivers = person_df[
                person_df['DRVR_LIC_TYPE_ID'].isin(VALID_LICENSE_CODES)
            ][['CRASH_ID', 'PRSN_NBR', 'UNIT_NBR']] 
            print(f"Número de personas con licencia válida: {len(licensed_drivers)}")
            
            #  Filtrar Units por Color Top 10 Y Estado Matrícula Top 25 
            qualifying_units = units_df[
                units_df['VEH_COLOR_ID'].isin(top_10_colors_list) &
                units_df['VEH_LIC_STATE_ID'].isin(top_25_state_abbrs_list) # Usamos la lista de abreviaturas
            ][['CRASH_ID', 'UNIT_NBR', 'VEH_MAKE_ID']] 
            print(f"Número de unidades con color Top 10 y estado matrícula Top 25: {len(qualifying_units)}")

            #  Unir 
            print("Iniciando uniones...")
            # Usar copias para evitar modificar DataFrames originales si hay merges repetidos
            merge1 = pd.merge(speeding_charges.copy(), licensed_drivers.copy(), on=['CRASH_ID', 'PRSN_NBR'], how='inner')
            print(f"  Registros después de unir Cargos y Personas: {len(merge1)}")
            
            final_merged_data = pd.merge(merge1, qualifying_units.copy(), on=['CRASH_ID', 'UNIT_NBR'], how='inner')
            print(f"  Registros después de unir con Unidades: {len(final_merged_data)}")

            #  Calcular Top 5 Marcas 
            if not final_merged_data.empty:
                make_counts = final_merged_data.groupby('VEH_MAKE_ID').size()
                make_counts_sorted = make_counts.sort_values(ascending=False)
                
                top_5_makes_final = make_counts_sorted.head(5)
                
                analysis7_result = top_5_makes_final.reset_index(name='QualifyingCount')
                analysis7_result.index = range(1, 1 + len(analysis7_result)) # Índice 1-5

                print(f"\nResultado del Análisis 7:")
                print("Top 5 Marcas de Vehículo que cumplen todos los criterios:")
                print(analysis7_result.to_string()) 

            else:
                print("\nResultado del Análisis 7:")
                print("No se encontraron registros que cumplan todas las condiciones después de las uniones.")

        except KeyError as e:
            print(f"ERROR: No se encontró la columna {e}. Verifica los nombres.")
        except Exception as e:
             print(f"Ocurrió un error inesperado durante el cálculo principal: {e}")
else:
    print("ERROR: Uno o más DataFrames o listas necesarias no fueron encontrados.")

print("\n--- Fin del Análisis 7 ---")


--- Iniciando Análisis 7 ---
Objetivo: Top 5 Marcas (Velocidad, Licencia Válida, Color Top 10, Estado Matrícula Top 25).
Número de cargos por velocidad/control encontrados: 25982
Número de personas con licencia válida: 132554
Número de unidades con color Top 10 y estado matrícula Top 25: 153993
Iniciando uniones...
  Registros después de unir Cargos y Personas: 46558
  Registros después de unir con Unidades: 44903

Resultado del Análisis 7:
Top 5 Marcas de Vehículo que cumplen todos los criterios:
  VEH_MAKE_ID  QualifyingCount
1        FORD             8126
2   CHEVROLET             7136
3      TOYOTA             4847
4       DODGE             3469
5      NISSAN             2963

--- Fin del Análisis 7 ---


# Análisis 8: "¿Qué tipo de vehículo tiene el mayor porcentaje de accidentes con lesiones graves/mortales?"

**Necesitamos:**
1. Filtrar `person_df` por `PRSN_INJRY_SEV_ID` en ['INCAPACITATING INJURY', 'KILLED'].
2. Unir con `units_df` para obtener el tipo de vehículo (`VEH_BODY_STYL_ID`).
3. Excluir tipos de vehículo no válidos: ['UNKNOWN', 'OTHER', 'NOT REPORTED'].
4. Calcular para cada tipo de vehículo:
   - Número total de accidentes (de `units_df`).
   - Número de accidentes graves/mortales (del filtro aplicado).
   - Porcentaje = (accidentes graves / total accidentes) * 100.
5. Ordenar descendientemente por porcentaje.

**Archivos:**
- Primary_Person.csv (`person_df`)
- Units.csv (`units_df`)

**Columnas clave:**
- `person_df`: CRASH_ID, UNIT_NBR, PRSN_INJRY_SEV_ID
- `units_df`: CRASH_ID, UNIT_NBR, VEH_BODY_STYL_ID

In [15]:
# Necesitamso los porcentajes de accidentes graves/mortales por tipo de vehículo

print("\n--- Análisis 8: % de Accidentes Graves/Mortales por Tipo de Vehículo ---")

SEVERE_CODES = ['INCAPACITATING INJURY', 'KILLED']
EXCLUDE_STYLES = ['UNKNOWN', 'OTHER  (EXPLAIN IN NARRATIVE)', 'NOT REPORTED']

if {'person_df', 'units_df'}.issubset(globals()):
    try:
        # Filtrar y unir datos 
        merged = (
            person_df[person_df['PRSN_INJRY_SEV_ID'].isin(SEVERE_CODES)]
            .merge(units_df[['CRASH_ID', 'UNIT_NBR', 'VEH_BODY_STYL_ID']], 
                 on=['CRASH_ID', 'UNIT_NBR'])
            .dropna(subset=['VEH_BODY_STYL_ID'])
            .query("VEH_BODY_STYL_ID not in @EXCLUDE_STYLES")
        )
        
        # Calcular conteos y porcentaje
        severe_counts = merged['VEH_BODY_STYL_ID'].value_counts().rename('SevereCount')
        total_counts = units_df['VEH_BODY_STYL_ID'].value_counts().rename('TotalCount')
        
        result = (
            pd.concat([severe_counts, total_counts], axis=1)
            .assign(Percentage=lambda x: (x['SevereCount'] / x['TotalCount'] * 100).round(2))
            .sort_values('Percentage', ascending=False)
            .head(15)
        )
        
        print("\nTop 15 Tipos de Vehículo por Letalidad (%):")
        print(result.to_string())
        
    except Exception as e:
         handle_dataframe_errors(e)


--- Análisis 8: % de Accidentes Graves/Mortales por Tipo de Vehículo ---

Top 15 Tipos de Vehículo por Letalidad (%):
                               SevereCount  TotalCount  Percentage
VEH_BODY_STYL_ID                                                  
MOTORCYCLE                           208.0         770       27.01
FARM EQUIPMENT                         3.0          44        6.82
PICKUP                               615.0       36508        1.68
FIRE TRUCK                             1.0          62        1.61
PASSENGER CAR, 2-DOOR                175.0       11027        1.59
PASSENGER CAR, 4-DOOR               1026.0       65504        1.57
SPORT UTILITY VEHICLE                501.0       33276        1.51
POLICE CAR/TRUCK                       6.0         398        1.51
TRUCK TRACTOR                         48.0        3274        1.47
VAN                                   65.0        5568        1.17
TRUCK                                 29.0        3641        0.80
YELLOW SCH